<a href="https://colab.research.google.com/github/ghadfield32/YOLO_exploration/blob/main/yolov10ex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


    Before you start
    Install YOLOv10
    Download pre-trained weights
    Download example data
    Inference with Pre-trained COCO Model
    Download dataset from Roboflow Universe
    Custom Training
    Validate Custom Model
    Inference with Custom Model


In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
import os
HOME = os.getcwd()
print(HOME)

!pip install -q supervision
!pip install -q git+https://github.com/THU-MIG/yolov10.git
!pip install roboflow
#!pip install ultralytics


/content
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
!mkdir -p {HOME}/weights
weights_urls = [
    "https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10n.pt",
    "https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10s.pt",
    "https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10m.pt",
    "https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10b.pt",
    "https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10x.pt",
    "https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10l.pt"
]
for url in weights_urls:
    !wget -P {HOME}/weights -q {url}
!ls -lh {HOME}/weights


total 652M
-rw-r--r-- 1 root root  80M May 23 09:38 yolov10b.pt
-rw-r--r-- 1 root root  80M May 23 09:38 yolov10b.pt.1
-rw-r--r-- 1 root root 100M May 23 09:38 yolov10l.pt
-rw-r--r-- 1 root root 240K May 27 13:17 yolov10m.pt
-rw-r--r-- 1 root root  64M May 23 09:38 yolov10m.pt.1
-rw-r--r-- 1 root root  11M May 23 09:38 yolov10n.pt
-rw-r--r-- 1 root root  11M May 23 09:38 yolov10n.pt.1
-rw-r--r-- 1 root root  32M May 23 09:38 yolov10s.pt
-rw-r--r-- 1 root root  32M May 23 09:38 yolov10s.pt.1
-rw-r--r-- 1 root root 123M May 23 09:38 yolov10x.pt
-rw-r--r-- 1 root root 123M May 23 09:38 yolov10x.pt.1


In [4]:
from ultralytics import YOLOv10

model = YOLOv10(f'{HOME}/weights/yolov10n.pt')
# results = model(source=f'/content/img.png', conf=0.25)

# print(results[0].boxes.xyxy) #bounding boxes
# print(results[0].boxes.conf) #confidence
# print(results[0].boxes.cls) #class


In [5]:

from roboflow import Roboflow
rf = Roboflow(api_key="htpcxp3XQh7SsgMfjJns")
project = rf.workspace("basketball-formations").project("basketball-and-hoop-7xk0h")
version = project.version(11)
dataset = version.download("yolov8")

# # Move the dataset to the datasets location
# import shutil

# source_dir = "/content/basketball-and-hoop-11"
# target_dir = "/content/datasets/basketball-and-hoop-11"
# os.makedirs("/content/datasets", exist_ok=True)
# shutil.move(source_dir, target_dir)


# Verify the dataset directory structure
!ls /content
!ls /content/basketball-and-hoop-11




loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.34, to fix: `pip install ultralytics==8.0.196`


Extracting Dataset Version Zip to basketball-and-hoop-11 in yolov8:: 100%|██████████| 5288/5288 [00:01<00:00, 4108.89it/s]


basketball-and-hoop-11	sample_data  weights
data.yaml  README.dataset.txt  README.roboflow.txt  test  train  valid


In [9]:
# Custom training
%cd {HOME}

!yolo task=detect mode=train epochs=1 batch=8 plots=True \
model={HOME}/weights/yolov10n.pt \
data={dataset.location}/data.yaml

!ls {HOME}/runs/detect/train/

/content
New https://pypi.org/project/ultralytics/8.2.22 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.34 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=/content/weights/yolov10n.pt, data=/content/basketball-and-hoop-11/data.yaml, epochs=1, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, val_period=1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None,

In [10]:
from IPython.display import Image
Image(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=600)
Image(filename=f'{HOME}/runs/detect/train/results.png', width=600)


FileNotFoundError: [Errno 2] No such file or directory: '/content/runs/detect/train/confusion_matrix.png'

Using this article to include the tracking:

https://supervision.roboflow.com/how_to/track_objects/#__tabbed_2_1

In [ ]:
#download the youtube video: https://www.youtube.com/watch?v=kh7s2tGvswc&t=3s
#to pull in the 3 point contest for devin booker
!pip install pytube

from pytube import YouTube

yt = YouTube('https://www.youtube.com/watch?v=kh7s2tGvswc&t=3s')
stream = yt.streams.filter(file_extension='mp4').first()
stream.download(output_path=HOME, filename='devin_booker_3pt_contest.mp4')




In [ ]:
import numpy as np
import supervision as sv
from ultralytics import YOLO

model = YOLO("yolov10n.pt")
tracker = sv.ByteTrack()
box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator()

def callback(frame: np.ndarray, _: int) -> np.ndarray:
    results = model(frame)[0]
    detections = sv.Detections.from_ultralytics(results)
    detections = tracker.update_with_detections(detections)

    labels = [
        f"#{tracker_id} {results.names[class_id]}"
        for class_id, tracker_id
        in zip(detections.class_id, detections.tracker_id)
    ]

    annotated_frame = box_annotator.annotate(frame.copy(), detections=detections)
    return label_annotator.annotate(annotated_frame, detections=detections, labels=labels)

sv.process_video(
    source_path=f"{HOME}/devin_booker_3pt_contest.mp4",
    target_path=f"{HOME}/result.mp4",
    callback=callback
)


In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/results.png', width=600)